# Model Exploration
This notebook is about the exploration of different regression model to choose the one who has the best metrics in order to create an end-to-end pipeline.

## Importation
In these cells, we import the necessary libraries and modules for model exploration, including custom functions for training and evaluating different regression models.

### EDA importations

In [ ]:

import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

### Model Importations

In [ ]:
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    explained_variance_score,
    median_absolute_error,
    mean_absolute_percentage_error
)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


### Models grid

In [ ]:
scoring_funcs = {
    "training_time" : 0,
    "prediction_time" : 0,
    "mean_absolute_error" : 0,
    "mean_squared_error" : 0,
    "r2_score" : 0,
    "explained_variance_score" : 0,
    "median_absolute_error" : 0,
    "mean_absolute_percentage_error" : 0
}

In [ ]:
grid_boosting = {
    "boosting_model__n_estimators": [100, 200, 300, 500],
    "boosting_model__learning_rate": [0.001, 0.01, 0.1, 0.2],
    "boosting_model__max_depth": [1, 3, 5, 7],
    "boosting_model__min_samples_split": [2, 5, 10],
    "boosting_model__min_samples_leaf": [1, 2, 4],
    "boosting_model__random_state": [42]
}

In [ ]:
grid_polinomial = {
    "degree": [2, 3, 4] # Bias always true, as we want to add intercept
}

In [ ]:
grid_regresor_tree = {
    "boosting_model__criterion": ["squared_error", "friedman_mse", "absolute_error"],
    "boosting_model__splitter": ["best", "random"],
    "boosting_model__max_depth": [None, 10, 20, 30],
    "boosting_model__min_samples_split": [2, 5, 10],
    "boosting_model__min_samples_leaf": [1, 2, 4],
    "boosting_model__random_state": [42]
}

In [ ]:
grid_polinomial_gridSearch = {
    "poly_features__degree": [2, 3, 4]  # Nota el nombre completo del paso
}

### Functions that we are going to use

In [2]:
def cross_validation_regression(conjunto, atributos, concepto, k=5, random=False, agregar_unos=False, undersample=False, oversample=False):
    if undersample:
        cantidad_menor_concepto = conjunto[concepto].value_counts().min()
        conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_menor_concepto, random_state=42) for valor in conjunto[concepto].unique()])

    if oversample:
        cantidad_mayor_concepto = conjunto[concepto].value_counts().max()
        conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_mayor_concepto, replace=True, random_state=42) for valor in conjunto[concepto].unique()])

    if random:
        conjunto = conjunto.sample(frac=1, random_state=11).reset_index(drop=True)

    x = conjunto[atributos]
    if agregar_unos:
        x.insert(0, "Unos", 1)

    y = conjunto[concepto]

    resultados = []
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    for train_index, test_index in kf.split(x):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        resultados.append((x_train, y_train, x_test, y_test))

    return resultados

def train_test_split_custom(conjunto, atributos, concepto, test_size=0.2, random_state=42):
    x = conjunto[atributos]
    y = conjunto[concepto]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)
    return x_train, y_train, x_test, y_test

In [3]:

def coding_categorical_columns(df):

    for column in df.columns:
        if len(df[column].unique()) <= 10 and df[column].dtype == 'object':
            unicos = df[column].unique()
            for i in range(len(unicos)):
                df[column] = df[column].replace({unicos[i]: i})
    return df

In [4]:
def categorical_and_numerical(df, features):
    categorical = []
    numerical = []

    for column in df[features].columns:
        if df[column].dtype == 'object':
            categorical.append(column)
        else:
            numerical.append(column)
    return categorical, numerical

In [ ]:
def validation_test(val_data, model, features, objetive_feature):
    x_val = val_data[features]
    y_val = val_data[objetive_feature]

    inicio = time.time()
    pred = model.predict(x_val)
    fin = time.time()
    prediction_time = fin - inicio

    mae = mean_absolute_error(y_val, pred)
    mse = mean_squared_error(y_val, pred)
    r2 = r2_score(y_val, pred)
    evs = explained_variance_score(y_val, pred)
    medae = median_absolute_error(y_val, pred)
    mape = mean_absolute_percentage_error(y_val, pred)

    results = {
        "prediction_time": prediction_time,
        "mean_absolute_error": mae,
        "mean_squared_error": mse,
        "r2_score": r2,
        "explained_variance_score": evs,
        "median_absolute_error": medae,
        "mean_absolute_percentage_error": mape
    }

    return results

In [ ]:
import matplotlib.pyplot as plt
def dataframe_to_png(df,output_path,title="Data Report",dpi=200,index_name="Index"):


    # Copiamos el DF para no modificar el original
    df_plot = df.copy()
    df_plot.insert(0, index_name, df_plot.index.astype(str))

    fig, ax = plt.subplots(
        figsize=(len(df_plot.columns) * 2.05, len(df_plot) * 0.5 + 1)
    )
    ax.axis("off")

    table = ax.table(
        cellText=df_plot.values,
        colLabels=df_plot.columns,
        loc="center",
        cellLoc="center"
    )

    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1, 1.5)

    # Estilo del header
    for (row, col), cell in table.get_celld().items():
        if row == 0:
            cell.set_text_props(weight="bold", color="white")
            cell.set_facecolor("#4F4F4F")

    plt.title(title, pad=20)
    plt.tight_layout()

    plt.savefig(output_path, dpi=dpi, bbox_inches="tight")
    plt.close()

### Training functions

In [ ]:
def scoring_k_folds(dict, folds, model, k_folds):

    for x_train, y_train, x_test, y_test in folds:

        inicio = time.time()
        model.fit(x_train, y_train)
        fin = time.time()
        dict["training_time"] += (fin - inicio) / k_folds

        inicio = time.time()
        pred = model.predict(x_test)
        fin = time.time()
        dict["prediction_time"] += (fin - inicio) / k_folds

        dict["mean_absolute_error"] += mean_absolute_error(y_test, pred) / k_folds
        dict["mean_squared_error"] += mean_squared_error(y_test, pred) / k_folds
        dict["r2_score"] += r2_score(y_test, pred) / k_folds
        dict["explained_variance_score"] += explained_variance_score(y_test, pred) / k_folds
        dict["median_absolute_error"] += median_absolute_error(y_test, pred) / k_folds
        dict["mean_absolute_percentage_error"] += mean_absolute_percentage_error(y_test, pred) / k_folds
        
    return dict, model

def scoring_grid(dict, model, x_test, y_test):

    inicio = time.time()
    pred = model.predict(x_test)
    fin = time.time()
    dict["prediction_time"] += (fin - inicio)

    dict["mean_absolute_error"] += mean_absolute_error(y_test, pred) 
    dict["mean_squared_error"] += mean_squared_error(y_test, pred) 
    dict["r2_score"] += r2_score(y_test, pred) 
    dict["explained_variance_score"] += explained_variance_score(y_test, pred) 
    dict["median_absolute_error"] += median_absolute_error(y_test, pred) 
    dict["mean_absolute_percentage_error"] += mean_absolute_percentage_error(y_test, pred)
        
    return dict, model

In [ ]:
def linear_regression_training(datos_train, features, objetive_feature, scoring_funcs, k_folds):
    linear_scoring = scoring_funcs.copy()

    categorical_features, numeric_features = categorical_and_numerical(datos_train, features)

    preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
    )

    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])
    
    folds = cross_validation_regression(datos_train, features, objetive_feature, k=k_folds, random=42)

    linear_scoring, model = scoring_k_folds(linear_scoring, folds, pipeline, k_folds)

    return linear_scoring, pipeline

In [ ]:




def polynomial_regression_grid_search(grid_split, grid, features, objetive_feature, k_folds, scoring_funcs):

    scoring_funcs_copy = scoring_funcs.copy()
    x_train = grid_split["x_train"]
    y_train = grid_split["y_train"]
    x_test = grid_split["x_test"]
    y_test = grid_split["y_test"]

    # Crear el pipeline con estandarización y características polinómicas
    categorical_features, numeric_features = categorical_and_numerical(x_train, features)

    preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
    )

    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("scaler", StandardScaler()),
        ("poly_features", PolynomialFeatures(include_bias=True)),
        ("regressor", LinearRegression())
    ])

    grid = GridSearchCV(pipeline, grid, cv=k_folds, scoring='r2', n_jobs=-1)
    inicio = time.time()
    grid.fit(x_train[features], y_train)
    fin = time.time()
    scoring_funcs_copy["training_time"] = fin - inicio

    print(grid.best_params_)
    print(grid.best_score_)
    best_model = grid.best_estimator_
    best_dict, model = scoring_grid(scoring_funcs_copy, best_model, x_test, y_test)

    return best_model, best_dict

In [ ]:
def boosting_regression_training_grid_search(grid_split, grid, features, objetive_feature, k_folds, scoring_funcs):

    scoring_funcs_copy = scoring_funcs.copy()
    x_train = grid_split["x_train"]
    y_train = grid_split["y_train"]
    x_test = grid_split["x_test"]
    y_test = grid_split["y_test"]

    categorical_features, numeric_features = categorical_and_numerical(x_train, features)

    preprocessor = ColumnTransformer(
    transformers=[
        #("num", StandardScaler(), numeric_features),
                ("num", "passthrough", numeric_features),

        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
    )

    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("boosting_model", GradientBoostingRegressor())
    ])

    grid = GridSearchCV(pipeline, grid, cv= k_folds, scoring='r2', n_jobs=-1)
    inicio = time.time()
    grid.fit(x_train[features], y_train)
    fin = time.time()
    scoring_funcs_copy["training_time"] = fin - inicio

    print(grid.best_params_)
    print(grid.best_score_)
    best_model = grid.best_estimator_
    best_dict, model =  scoring_grid(scoring_funcs_copy, best_model, x_test, y_test)

    return best_model, best_dict

In [ ]:
def decision_tree_regression_training_grid_search(grid_split, grid, features, objetive_feature, k_folds, scoring_funcs):

    scoring_funcs_copy = scoring_funcs.copy()
    x_train = grid_split["x_train"]
    y_train = grid_split["y_train"]
    x_test = grid_split["x_test"]
    y_test = grid_split["y_test"]

    categorical_features, numeric_features = categorical_and_numerical(x_train, features)

    preprocessor = ColumnTransformer(
    transformers=[
        #("num", StandardScaler(), numeric_features),
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
    )

    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("boosting_model", DecisionTreeRegressor())
    ])
    grid = GridSearchCV(pipeline, grid, cv= k_folds, scoring='r2', n_jobs=-1)
    inicio = time.time()
    grid.fit(x_train[features], y_train)
    fin = time.time()
    scoring_funcs_copy["training_time"] = fin - inicio

    print(grid.best_params_)
    print(grid.best_score_)
    best_model = grid.best_estimator_
    best_dict, model =  scoring_grid(scoring_funcs_copy, best_model, x_test, y_test)

    return best_model, best_dict

### Start of the notebook

In [ ]:
datos = pd.read_csv("../data/raw/dataset.csv")


In [5]:
features = datos.columns[:-1]
objetive_feature = datos.columns[-1]
#final_df = coding_categorical_columns(datos.copy())
datos_train, datos_val = train_test_split(datos, test_size=0.1, random_state=42)
categorical , numerical = categorical_and_numerical(datos, features)
print(categorical)
print(numerical)

['sex', 'smoker', 'region']
['age', 'bmi', 'children']


In [85]:
resultados = {}
modelos = {}

x_train, y_train, x_test, y_test = train_test_split_custom(datos_train, features, objetive_feature, test_size=0.1, random_state=42)

grid_splits = {
    "x_train": x_train,
    "y_train": y_train,
    "x_test": x_test,
    "y_test": y_test
}

k_folds = 5

linear_scoring, linear_model = linear_regression_training(datos_train, features, objetive_feature, scoring_funcs, k_folds=5)
resultados["LinearRegression"] = linear_scoring
modelos["LinearRegression"] = linear_model

print("Linear Regression Grid Search Results ready")

poly_model, poly_scoring = polynomial_regression_grid_search(grid_splits, grid_polinomial_gridSearch, features, objetive_feature, k_folds, scoring_funcs)
resultados["PolynomialRegression"] = poly_scoring
modelos["PolynomialRegression"] = poly_model

print("Polynomial Regression Grid Search Results ready")

boosting_model, boosting_scoring = boosting_regression_training_grid_search(grid_splits, grid_boosting, features, objetive_feature, k_folds, scoring_funcs)
resultados["BoostingRegression"] = boosting_scoring
modelos["BoostingRegression"] = boosting_model

print("Boosting Regression Grid Search Results ready")

tree_model, tree_scoring = decision_tree_regression_training_grid_search(grid_splits, grid_regresor_tree, features, objetive_feature, k_folds, scoring_funcs)
resultados["TreeRegression"] = tree_scoring
modelos["TreeRegression"] = tree_model

print("Decision Tree Regression Grid Search Results ready")



Linear Regression Grid Search Results ready
{'poly_features__degree': 2}
0.8295325540915357
Polynomial Regression Grid Search Results ready
{'boosting_model__learning_rate': 0.01, 'boosting_model__max_depth': 3, 'boosting_model__min_samples_leaf': 2, 'boosting_model__min_samples_split': 5, 'boosting_model__n_estimators': 500, 'boosting_model__random_state': 42}
0.8527951148850784
Boosting Regression Grid Search Results ready
{'boosting_model__criterion': 'absolute_error', 'boosting_model__max_depth': None, 'boosting_model__min_samples_leaf': 2, 'boosting_model__min_samples_split': 10, 'boosting_model__random_state': 42, 'boosting_model__splitter': 'random'}
0.8280386379156723
Decision Tree Regression Grid Search Results ready


In [94]:
# Asegurarse de que todos los valores de 'resultados' sean diccionarios planos
resultados_limpios = {}
for k, v in resultados.items():
    if isinstance(v, dict):
        # Convertir los tiempos a segundos (si no lo están) y redondear a 4 decimales
        v["training_time"] = round(float(v.get("training_time", 0)), 4)
        v["prediction_time"] = round(float(v.get("prediction_time", 0)), 4)
        resultados_limpios[k] = v
    else:
        print(f"Advertencia: el resultado de {k} no es un diccionario y será omitido. Tipo: {type(v)}")

if not resultados_limpios:
    print("No hay resultados válidos para mostrar.")
else:
    df_resultados = pd.DataFrame(resultados_limpios).T
    print(df_resultados)

                      training_time  prediction_time  mean_absolute_error  \
LinearRegression             0.0143           0.0053          4227.845742   
PolynomialRegression        19.2019           0.0061          2766.928928   
BoostingRegression         504.8198           0.0070          2084.936186   
TreeRegression              10.6018           0.0060          1498.776920   

                      mean_squared_error  r2_score  explained_variance_score  \
LinearRegression            3.780176e+07  0.742746                  0.743325   
PolynomialRegression        1.878467e+07  0.818895                  0.820980   
BoostingRegression          1.209358e+07  0.883405                  0.887273   
TreeRegression              1.475411e+07  0.857754                  0.861467   

                      median_absolute_error  mean_absolute_percentage_error  
LinearRegression                2500.356190                        0.421696  
PolynomialRegression            1848.401027              

In [95]:
df_resultados

,training_time,prediction_time,mean_absolute_error,mean_squared_error,r2_score,explained_variance_score,median_absolute_error,mean_absolute_percentage_error
LinearRegression,0.0143,0.0053,4227.845742,3.780176e+07,0.742746,0.743325,2500.356190,0.421696
PolynomialRegression,19.2019,0.0061,2766.928928,1.878467e+07,0.818895,0.820980,1848.401027,0.304907
BoostingRegression,504.8198,0.0070,2084.936186,1.209358e+07,0.883405,0.887273,1570.396715,0.291330
TreeRegression,10.6018,0.0060,1498.776920,1.475411e+07,0.857754,0.861467,404.610000,0.097027


In [90]:
import joblib
from pathlib import Path
# Definición centralizada de rutas



def save_model(model, model_name, base_dir=None):
    model_path = f"{model_name}.pkl"

    # Save the model
    joblib.dump(model, model_path)

    print(f"Modelo guardado en: {model_path}")

In [91]:
for model_name, model in modelos.items():
    save_model(model, model_name)

Modelo guardado en: LinearRegression.pkl
Modelo guardado en: PolynomialRegression.pkl
Modelo guardado en: BoostingRegression.pkl
Modelo guardado en: TreeRegression.pkl


In [96]:
resultados_val = {}


for model in modelos:
    print(f"Validating model: {model}")
    resultados_val[model] = validation_test(datos_val, modelos[model], features, objetive_feature)
    print(f"Results for {model}: {resultados_val[model]}")

resultados_limpios = {}
for k, v in resultados_val.items():
    if isinstance(v, dict):
        # Convertir los tiempos a segundos (si no lo están) y redondear a 4 decimales
        v["prediction_time"] = round(float(v.get("prediction_time", 0)), 4)
        resultados_limpios[k] = v
    else:
        print(f"Advertencia: el resultado de {k} no es un diccionario y será omitido. Tipo: {type(v)}")

if not resultados_limpios:
    print("No hay resultados válidos para mostrar.")
else:
    df_resultados_val = pd.DataFrame(resultados_limpios).T
    df_resultados_val

Validating model: LinearRegression
Results for LinearRegression: {'prediction_time': 0.0070018768310546875, 'mean_absolute_error': 4142.558205549579, 'mean_squared_error': 32139784.21268442, 'r2_score': 0.7670275482907529, 'explained_variance_score': 0.7702944669970923, 'median_absolute_error': 2566.8450595182417, 'mean_absolute_percentage_error': 0.4596303992863332}
Validating model: PolynomialRegression
Results for PolynomialRegression: {'prediction_time': 0.0060176849365234375, 'mean_absolute_error': 2760.40992742727, 'mean_squared_error': 19789939.22121045, 'r2_score': 0.856548176271738, 'explained_variance_score': 0.8584441977213703, 'median_absolute_error': 1819.520284275466, 'mean_absolute_percentage_error': 0.31656194149999}
Validating model: BoostingRegression
Results for BoostingRegression: {'prediction_time': 0.0070095062255859375, 'mean_absolute_error': 2370.8266008309624, 'mean_squared_error': 17201322.85788038, 'r2_score': 0.8753123439683483, 'explained_variance_score': 0

In [97]:
df_resultados_val

,prediction_time,mean_absolute_error,mean_squared_error,r2_score,explained_variance_score,median_absolute_error,mean_absolute_percentage_error
LinearRegression,0.007,4142.558206,3.213978e+07,0.767028,0.770294,2566.845060,0.459630
PolynomialRegression,0.006,2760.409927,1.978994e+07,0.856548,0.858444,1819.520284,0.316562
BoostingRegression,0.007,2370.826601,1.720132e+07,0.875312,0.876424,1684.325699,0.344932
TreeRegression,0.007,1660.434125,2.023910e+07,0.853292,0.858478,371.296325,0.127930


In [101]:
dataframe_to_png(df_resultados, "training_results.png", title="Training Results", index_name="Model")
dataframe_to_png(df_resultados_val, "scoring_results.png", title="Validation Results", index_name="Model")

In [ ]:
def validation_test(val_data, model, features, objetive_feature):
    x_val = val_data[features]
    y_val = val_data[objetive_feature]

    pred = model.predict(x_val)

    df_results = pd.DataFrame({
        "Actual": y_val,
        "Predicted": pred
    })

    return df_results

In [ ]:
resultados = {}


for model in modelos:
    print(f"Validating model: {model}")
    resultados[model] = validation_test(datos_val, modelos[model], features, objetive_feature)

Validating model: LinearRegression
Validating model: PolynomialRegression
Validating model: BoostingRegression
Validating model: TreeRegression


In [ ]:
resultados["BoostingRegression"]

,Actual,Predicted
764,9095.06825,10458.013296
887,5272.17580,5868.237333
890,29330.98315,27930.037321
1293,9301.89355,9856.327037
259,33750.29180,34218.099890
...,...,...
358,1837.28190,4075.198454
551,3972.92470,4591.331513
778,5934.37980,7844.541832
1127,5836.52040,7871.011392


In [ ]:
resultados["TreeRegression"]

,Actual,Predicted
764,9095.06825,8606.217400
887,5272.17580,5266.365600
890,29330.98315,28950.469200
1293,9301.89355,8944.115100
259,33750.29180,34439.855900
...,...,...
358,1837.28190,1824.285400
551,3972.92470,3994.177800
778,5934.37980,6184.299400
1127,5836.52040,6184.299400


In [ ]:
resultados["LinearRegression"]

,Actual,Predicted
764,9095.06825,9326.582483
887,5272.17580,7281.152483
890,29330.98315,36748.190150
1293,9301.89355,9548.708581
259,33750.29180,26709.810704
...,...,...
358,1837.28190,7178.424890
551,3972.92470,5406.662149
778,5934.37980,9087.184349
1127,5836.52040,9371.547137


In [ ]:
resultados["PolynomialRegression"]

,Actual,Predicted
764,9095.06825,10998.179596
887,5272.17580,6041.728581
890,29330.98315,33917.089467
1293,9301.89355,11221.528590
259,33750.29180,28484.511446
...,...,...
358,1837.28190,2291.424640
551,3972.92470,5497.677227
778,5934.37980,7952.362340
1127,5836.52040,7628.337188
